In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import shutil
import sys

from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import datasets
from utils.model import AugLayer, ClassificationModel, CheckpointCallback, CustomLRDecay, plot_history

In [ ]:
BATCH_SIZE = 2048
EPOCHS = 100
PATH_CHECKPOINT = './CIFAR10/checkpoint'
PATH_CSV = './CIFAR10/training.log'
LR_PROFILE = [1e-4, 1e-2, 1e-3, 1e-4, 1e-5]
LR_EPOCHS = [5, 25, 60, 85]

AUG_ROT = 20
AUG_ZOOM = 0.15
AUG_SHIFT = 0.2
AUG_SHEAR = 0.15
AUG_BRIGHTNESS = [0.9, 1.1]
AUG_FLIP = 'horizontal_and_vertical'

shutil.rmtree(PATH_CHECKPOINT, ignore_errors=True)
os.makedirs(PATH_CHECKPOINT)

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train, X_test = X_train/255., X_test/255.

In [ ]:
model = ClassificationModel((32,32,3), 10)

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [4]:
!pip show tensorflow

In [5]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
